In [1]:
# System Imports
import os
import requests
import json 
import time
import datetime

# API Tokens 
# Add api keys to Heroku config vars to access them
# Using Openweather API
open_weather_token = os.environ.get('WEATHER_KEY')

# DS Logic imports
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import feedparser

In [3]:
def get_modis_data():
    modis_url = "https://firms.modaps.eosdis.nasa.gov/data/active_fire/c6/csv/MODIS_C6_USA_contiguous_and_Hawaii_24h.csv"
    modis_data = pd.read_csv(modis_url)
    return modis_data

def process_live_data(original_df):
    """
    Pre processes live data to match pipeline expectations.
    """
    print("process_live_data!")
    df = original_df.copy()
    # process satellite labels
    df["satellite"] = df["satellite"].replace({"T": "Terra", "A": "Aqua"})

    # process time features
    df["acq_time"] = (df["acq_time"] // 100) * 60 + (df["acq_time"] % 100)
    df["timestamp"] = df.apply(
        lambda x: datetime.datetime.strptime(x["acq_date"], "%Y-%m-%d")
        + datetime.timedelta(minutes=x["acq_time"]),
        axis=1,
    )
    # df['version'] = df['version'].apply(str)
    df["month"] = df["timestamp"].dt.month
    df["week"] = df["timestamp"].dt.weekofyear
    df.drop(columns=["acq_date", "acq_time", "timestamp"], inplace=True)

    return df

In [13]:
from functions import (
    get_weather, get_modis_data, reduce_points, 
    add_weather_columns, populate_weather, label_fires,
    process_live_data, clean_df, fires_list
    )
import time
import psycopg2
import urllib.parse as up
import datetime
import os

import feedparser

In [6]:
# Get the modis data
df = get_modis_data()
df = process_live_data(df)

process_live_data!


In [7]:
# Reduce number of points
df = reduce_points(df)

# Add empty weather columns
df = add_weather_columns(df)

/Users/Eric/Lambda/Labs/forest-fire-watch-ds/Getter_Api/app/functions.py:74: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  coords = df.as_matrix(columns=['lat', 'lon'])


In [18]:
# Label data
def label_fires(df):
    # Instantiate labels list
    labels = []
    
    # Get lats and lons from df
    lats = df['lat'].tolist()
    lons = df['lon'].tolist()
    
    # Pull confirmed fires
    fires = fires_list()
    locations = [entry['location'] for entry in fires]
    
    # loop data points
    for n in range(len(lats)):

        # Get position
        lat = lats[n]
        lon = lons[n]

        # loop fires
        for fire in locations:

            # Get fire position
            fire_lat = fire[1]
            fire_lon = fire[0]

            # Impose domain restraints on latitude
            if -90 <= lat <= 90:
                pass
            else:
                print('yee')
            if -90 <= fire_lat <= 90:
                pass
            else:
                print(fire_lat)

            distance = great_circle((lat, lon), (fire_lat, fire_lon)).miles # haversine(lons[n], lats[n], fire[1], fire[0])
            label = 0
            if distance < 0.3:
                label = 1
                labels.append(label)
                break
            else:
                pass

        if label != 1:
            labels.append(label)
            
    # append labels to df
    labelled_df = df.copy()
    labelled_df['fire'] = labels
    
    return labelled_df

In [19]:
# Label the data
print('labeling fires...')
df = label_fires(df)
print('done!')

labeling fires...
done!


In [21]:
fires_list()

[{'name': 'Martinez 3 Fire (Wildfire)', 'location': (-116.14222222222, 33.64)},
 {'name': 'Brushy Mountain/Buskirk Prescribed Burn (Prescribed Fire)',
  'location': (-80.228888888889, 37.832777777778)},
 {'name': '2019 Malheur NF Prescribed Fire (Prescribed Fire)',
  'location': (-118.95138888889, 44.417777777778)},
 {'name': 'Taboose Fire (Wildfire)',
  'location': (-118.34527777778, 37.034166666667)},
 {'name': 'Ranch Fire (Wildfire)',
  'location': (-122.60944444444, 40.040277777778)},
 {'name': 'Kaibab NF South Zone Rx Fire (Prescribed Fire)',
  'location': (-112.19805555556, 35.290833333333)},
 {'name': 'Hillside Fire (Wildfire)',
  'location': (-117.28527777778, 34.183333333333)},
 {'name': 'Decker Fire (Wildfire)',
  'location': (-106.00388888889, 38.409444444444)},
 {'name': '2019 Fall Rx Burns for Flagstaff Dist. (Prescribed Fire)',
  'location': (-111.70972222222, 35.1125)},
 {'name': 'Caples Fire (Wildfire)',
  'location': (-120.14527777778, 38.723888888889)},
 {'name': 'San